In [ ]:
%%time

!pip install -q -U simpletransformers

CPU times: user 63.9 ms, sys: 55 ms, total: 119 ms
Wall time: 9.44 s


In [ ]:
import pandas as pd
import seaborn as sns
import transformers
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import os, json, gc, re, random
from sklearn.preprocessing import LabelEncoder

In [ ]:
import torch, transformers, tokenizers
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
from torch.nn import BCEWithLogitsLoss, BCELoss
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score

In [ ]:
movie_df = pd.read_csv("/content/kinopoisk-top250g.csv", sep=',')

movies = movie_df


In [ ]:
shortlisted_genres = movies["Genre"].value_counts().reset_index(name="count").query("count > 15")["index"].tolist()
movies = movies[movies["Genre"].isin(shortlisted_genres)].reset_index(drop=True)

In [ ]:
movies['GenreCorrected'] =movies['Genre'] 
movies['GenreCorrected']=movies['GenreCorrected'].str.strip()
movies['GenreCorrected']=movies['GenreCorrected'].str.replace('драма', 'Драма')
movies['GenreCorrected']=movies['GenreCorrected'].str.replace('Драма ', 'Драма')
movies['GenreCorrected']=movies['GenreCorrected'].str.replace('Детективный', 'Детектив')

In [ ]:
movies['GenreCorrected'].unique()

array(['Драма', 'Военный', 'Комедия', 'Криминал', 'Романтика'],
      dtype=object)

In [ ]:
movies = movies.sample(frac=1).reset_index(drop=True)

In [ ]:
movies

,rating,Genre,movie,year,country,rating_ball,Plot,director,screenwriter,actors,url_logo,GenreCorrected
0,0,Драма,Побег из Шоушенка,1994,США,9.111,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг,Тим Роббинс; Морган Фриман; Боб Гантон; Уильям...,'https://st.kp.yandex.net/images/film_iphone/i...,Драма
1,126,Комедия,Как украсть миллион,1966,США,8.204,Действие происходит в Париже. Молодая девушка;...,Уильям Уайлер,Гарри Кернитц; Джордж Брэдшоу,Одри Хепберн; Питер О’Тул; Илай Уоллак; Хью Гр...,'https://st.kp.yandex.net/images/film_iphone/i...,Комедия
2,2,Драма,Форрест Гамп,1994,США,8.913,От лица главного героя Форреста Гампа; слабоум...,Роберт Земекис,Эрик Рот; Уинстон Грум,Том Хэнкс; Робин Райт; Салли Филд; Гэри Синиз;...,'https://st.kp.yandex.net/images/film_iphone/i...,Драма
3,206,Драма,Хулиганы,2005,США,8.026,Студент факультета журналистики Мэтт Бакнер от...,Лекси Александр,Лекси Александр; Дуги Бримсон; Джош Шелов,Элайджа Вуд; Чарли Ханнэм; Клэр Форлани; Марк ...,'https://st.kp.yandex.net/images/film_iphone/i...,Драма
4,124,Драма,Город Бога,2002,Бразилия,8.018,Фильм охватывает события; происходящие на прот...,Фернанду Мейреллиш; Катя Лунд,Браулио Мантовани; Паолу Линс,Алешандре Родригис; Леандру Фирмину; Феллипе Х...,'https://st.kp.yandex.net/images/film_iphone/i...,Драма
...,...,...,...,...,...,...,...,...,...,...,...,...
131,131,Драма,Афера,1973,США,8.138,В 1930-е годы двое отпетых мошенников пытаются...,Джордж Рой Хилл,Дэвид С. Уорд,Пол Ньюман; Роберт Редфорд; Роберт Шоу; Чарльз...,'https://st.kp.yandex.net/images/film_iphone/i...,Драма
132,86,Комедия,Укрощение строптивого,1980,Италия,8.384,Категорически не приемлющий женского общества ...,Франко Кастеллано; Джузеппе Моччиа,Франко Кастеллано; Джузеппе Моччиа; Уильям ...,Адриано Челентано; Орнелла Мути; Эдит Питерс; ...,'https://st.kp.yandex.net/images/film_iphone/i...,Комедия
133,129,Драма,Легенда о пианисте,1998,Италия,8.138,В первый день XX века на пароходе «Вирджиния» ...,Джузеппе Торнаторе,Алессандро Барикко; Джузеппе Торнаторе,Тим Рот; Прюитт Тэйлор Винс; Мелани Тьерри; Би...,'https://st.kp.yandex.net/images/film_iphone/i...,Драма
134,76,Военный,Офицеры,1971,СССР,8.452,Судьбы двух друзей — Алексея Трофимова и Ивана...,Владимир Роговой,Кирилл Рапопорт; Борис Васильев,Георгий Юматов; Василий Лановой; Алина Покровс...,'https://st.kp.yandex.net/images/film_iphone/i...,Военный


In [ ]:
movies['genre_new'] = [x.replace('|',',').split(',') for x in movies['GenreCorrected'].values]
movies['genre_new'] = movies['genre_new'].apply(lambda genres: [x.strip() for x in genres])
movies['genre_new'] = movies['genre_new'].apply(lambda genres: list(filter(lambda x: len(x) != 0, genres)))
movies['genre_new'].iloc[30]

['Военный']

In [ ]:
movies = movies[["Plot", "GenreCorrected", "genre_new"]]

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
genre_encoded = multilabel_binarizer.fit_transform(movies['genre_new'])

values = movies[["Plot", "GenreCorrected", "genre_new"]].values
values = [[values[i][0], values[i][1], values[i][2], genre_encoded[i]] for i in range(len(values)) ]

movies = pd.DataFrame(values, columns=['text', 'GenreCorrected', "genre_new", "labels"])

In [ ]:
len(movies), len(multilabel_binarizer.classes_)

(136, 5)

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs

args = MultiLabelClassificationArgs()
args.reprocess_input_data = True
args.overwrite_output_dir = True
args.save_model_every_epoch = False
args.save_model_every_epoch = False
args.max_seq_length = 80
args.train_batch_size = 60
args.num_train_epochs = 15
args.threshold = 0.26


# Create a ClassificationModel
model = MultiLabelClassificationModel('bert', 'bert-base-cased', num_labels=len(multilabel_binarizer.classes_), args=args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not in

In [ ]:
train_df, eval_df = train_test_split(movies, test_size=0.3, random_state=42)
train_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)
print("train: ", len(train_df))
print("evaluation: ", len(eval_df))

train:  95
evaluation:  41


In [ ]:
%%time
model.train_model(train_df[["text", "labels"]])

  0%|          | 0/95 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 1 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 2 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

...

Running Epoch 12 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 13 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 14 of 15:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 6.86 s, sys: 890 ms, total: 7.75 s
Wall time: 8.95 s


(30, 0.5205824623505274)

In [ ]:
result, model_outputs, wrong_output = model.eval_model(eval_df[["text", "labels"]])

  0%|          | 0/41 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# https://scikit-learn.org/stable/modules/model_evaluation.html#label-ranking-average-precision
print(result)

{'LRAP': 0.6178861788617885, 'eval_loss': 0.49529142181078595}


In [ ]:
corr = 0
for i in range(len(eval_df)):
    if ((model_outputs[i] > 0.3).astype(int) == np.array(eval_df['labels'][i])).all():
        corr+=1
print("acc: {}".format(corr / len(eval_df)))

acc: 0.3902439024390244


In [ ]:
for _ in range(15):
    random_idx = random.randint(0, len(eval_df)-1)
    text = eval_df.iloc[random_idx]['text']
    true_genre = eval_df.iloc[random_idx]['GenreCorrected']
    predicted_genre_encoded, _ = model.predict([text])
    predicted_genre = multilabel_binarizer.inverse_transform(np.array(predicted_genre_encoded))

    print('\nTrue Genre: {}\n'.format(true_genre))
    print('Predicted Genre: {}\n'.format(predicted_genre))
    print('Plot: {}\n'.format(text))
    print("-------------------------------------------")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


True Genre: Комедия

Predicted Genre: [('Драма',)]

Plot: История о злоключениях извечного любимца публики; маленького бродяги; который на этот раз отправляется на золотые прииски Аляски и как всегда порадует вас бесконечным калейдоскопом трюков и комических ситуаций.

-------------------------------------------


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]


True Genre: Драма

Predicted Genre: [('Военный', 'Драма', 'Комедия')]

Plot: Рассказ о нелегких буднях учительницы английского языка; преподающей в одной из школ калифорнийского городка Лонг-Бич. Ее ученики — почти сплошь субъекты; для которых английский совсем не является родным языком. Ко всему прочему; Лонг-Бич славится своими бандитскими традициями.

-------------------------------------------


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


True Genre: Комедия

Predicted Genre: [('Драма',)]

Plot: В легендах и мифах есть персонажи; главная задача которых — исполнять желания. У арабов это — джинны; у ирландцев — лепреконы; у китайцев — драконы; у европейцев — феи и лесные духи. А в Америке есть некто О. Ж. Грант — довольно странный и забавный человек; который тоже может исполнить любое желание. Но будьте осторожны; он очень проказлив! Вот его-то и встречает однажды Нил Оливер.Нил вполне доволен своей жизнью: у него обеспеченные родители; симпатичная невеста и впереди блестящая карьера юриста. Но с недавних пор по ночам ему стала сниться загадочная девушка; которую он никак не может выбросить из головы.Чудаковатый Грант приглашает Нила совершить поездку по таинственной автостраде 60; которой нет ни на одной карте США. И Нил бросает всё и пускается в самое невероятное и рискованное путешествие в своей жизни; решив во что бы то ни стало разыскать незнакомку из своих снов.

-------------------------------------------
